Calculating Recency

In [ ]:
df['last_purchase_date'] = df.groupby('CustomerId')['Created_At'].transform(max)
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date']).dt.date

In [ ]:
df['last_purchase_date'] = pd.to_datetime(df['last_purchase_date'])
df['invoicedate'] = pd.to_datetime(df['Created_At'])

In [ ]:
df['ref_date'] = df['invoicedate'].max() + timedelta(days=7)
df['ref_date'] = df['ref_date'].dt.date
df['ref_date'] = pd.to_datetime(df['ref_date'])
df['date'] = pd.to_datetime(df['invoicedate'])
df['date'] = df['date'].dt.date
df.sample(10)

In [ ]:
customer_recency = pd.DataFrame(df.groupby('CustomerId', as_index=False).date.max())
customer_recency.head()

In [ ]:
df["customer_recency"] = df["ref_date"] - df["last_purchase_date"]
df[["CustomerId", 'last_purchase_date', "ref_date", "customer_recency"]]

In [ ]:
df['recency2'] = pd.to_numeric(df['customer_recency'].dt.days.astype('int64'))
df[["CustomerId", 'last_purchase_date', "ref_date", "customer_recency", 'recency2']]

In [ ]:
customer_recency = df.groupby('CustomerId', as_index=False)['recency2'].mean()
customer_recency.rename(columns={'recency2':'Recency'}, inplace=True)
customer_recency.sort_values(by='Recency', ascending=False).head()

Calculating Frequency

In [ ]:
customer_frequency = df_copy.groupby('CustomerId', as_index=False)['orderIncrementId'].nunique()
customer_frequency.rename(columns={'orderIncrementId':'Frequency'}, inplace=True)
customer_frequency.sort_values(by='Frequency', ascending=False)

Calculating Monetary

In [ ]:
customer_monetary = df_copy.groupby('CustomerId', as_index=False)['Sum of PriceInclTax'].sum()
customer_monetary.rename(columns={'Sum of PriceInclTax':'Monetary'}, inplace=True)
customer_monetary.sort_values(by='Monetary', ascending=False).head()

delete outliers

In [ ]:
Q1 = customer_rfm['Monetary'].quantile(0.25)
Q3 = customer_rfm['Monetary'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


customer_rfm = customer_rfm[(customer_rfm['Monetary'] >= lower_bound) & (customer_rfm['Monetary'] <= upper_bound)]


Q1 = customer_rfm['Frequency'].quantile(0.25)
Q3 = customer_rfm['Frequency'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


customer_rfm = customer_rfm[(customer_rfm['Frequency'] >= lower_bound) & (customer_rfm['Frequency'] <= upper_bound)]



Creating RFM Table

In [ ]:
customer_rfm = pd.merge(pd.merge(customer_recency, customer_frequency, on='CustomerId'), customer_monetary, on='CustomerId')
customer_rfm.head()

In [ ]:
quantiles = customer_rfm.quantile(q = [0.25, 0.50, 0.75])
quantiles

In [ ]:
def recency_scoring(rfm):
    if rfm.Recency <= #put value of 0.25 quantiles
        recency_score = 4
    elif rfm.Recency <= #put value of 0.50 quantiles
        recency_score = 3
    elif rfm.Recency <= #put value of 0.75 quantiles
        recency_score = 2
    else:
        recency_score = 1
    return recency_score

customer_rfm['Recency_Score'] = customer_rfm.apply(recency_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
def frequency_scoring(rfm):
    if rfm.Frequency >= #put value of quantiles
        frequency_score = 2
    else:
        frequency_score = 1
    return frequency_score

customer_rfm['Frequency_Score'] = customer_rfm.apply(frequency_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
def frequency_scoring(rfm):
    if rfm.Frequency >= #put value of quantiles
        frequency_score = 2
    else:
        frequency_score = 1
    return frequency_score

customer_rfm['Frequency_Score'] = customer_rfm.apply(frequency_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
def monetary_scoring(rfm):
    if rfm.Monetary >= #put value of 0.75 quantiles
        monetary_score = 4
    elif rfm.Monetary >= #put value of 0.50 quantiles
        monetary_score = 3
    elif rfm.Monetary >= #put value of 0.25 quantiles
        monetary_score = 2
    else:
        monetary_score = 1
    return monetary_score

customer_rfm['Monetary_Score'] = customer_rfm.apply(monetary_scoring, axis=1)
customer_rfm.sample(10)

In [ ]:
def rfm_scoring(customer):
    return str(int(customer['Recency_Score'])) + str(int(customer['Frequency_Score'])) + str(int(customer['Monetary_Score']))


customer_rfm['Customer_RFM_Score'] = customer_rfm.apply(rfm_scoring, axis=1)
customer_rfm.sample(8)

In [ ]:
customer_rfm['RFM_Label'] = customer_rfm['Recency_Score'] + customer_rfm['Frequency_Score'] + customer_rfm['Monetary_Score']

customer_rfm.sample(8)

In [ ]:
segments = {'Customer_Segment':['Champion',
                                'Top Loyal Customer',
                                'Loyal Customer',
                                'Top Recent Customer',
                                'Recent Customer',
                                'Top Customer Needed Attention',
                                'Customer Needed Attention',
                                'Top Lost Customer',
                                'Lost Customer'],
            'RFM':['(2|3|4)-(4)-(4)',
                   '(3)-(1|2|3|4)-(3|4)',
                   '(3)-(1|2|3|4)-(1|2)',
                   '(4)-(1|2|3|4)-(3|4)',
                   '(4)-(1|2|3|4)-(1|2)',
                   '(2|3)-(1|2|3|4)-(3|4)',
                   '(2|3)-(1|2|3|4)-(1|2)',
                   '(1)-(1|2|3|4)-(3|4)',
                   '(1)-(1|2|3|4)-(1|2)',]}

pd.DataFrame(segments)

In [ ]:
def categorizer(rfm):

    if (rfm[0] in ['2', '3', '4']) & (rfm[1] in ['4']) & (rfm[2] in ['4']):
        rfm = 'Champion'

    elif (rfm[0] in ['3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Loyal Customer'

    elif (rfm[0] in ['3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Loyal Customer'

    elif (rfm[0] in ['4']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Recent Customer'

    elif (rfm[0] in ['4']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Recent Customer'

    elif (rfm[0] in ['2', '3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Customer Needed Attention'

    elif (rfm[0] in ['2', '3']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Customer Needed Attention'

    elif (rfm[0] in ['1']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['3', '4']):
        rfm = 'Top Lost Customer'

    elif (rfm[0] in ['1']) & (rfm[1] in ['1', '2', '3', '4']) & (rfm[2] in ['1', '2']):
        rfm = 'Lost Customer'

    return rfm

In [ ]:
customer_rfm['Customer_Category'] = customer_rfm["Customer_RFM_Score"].apply(categorizer)
customer_rfm